# Message Functional Test

https://www.analog.com/media/en/technical-documentation/data-sheets/adau1401.pdf

In [1]:
from pathfinder import Pathfinder

# abs_paths = ['']
# relative_paths = Pathfinder.relative_paths_from_abs(abs_paths)

relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']]

Pathfinder.append_relative_paths(relative_paths)

In [2]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [3]:
from bridges.ftdi.adapters.micropython import machine
from bridges.ftdi.controllers.i2c import I2cController
from sigma.sigma_dsp.adau import ADAU1401
from sigma.bus import adapters

## Debug mode?

In [4]:
cls = ADAU1401

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## DSP processor

In [5]:
with_hardware_device = False  # set True if hardware is connected.

if with_hardware_device:
    
    ctrl = I2cController()
    _machine = ctrl.get_gpio()

    _i2c = ctrl.I2C()

    _pin_reset = _machine.Pin('ADBUS4', mode = machine.Pin.OUT)
    _pin_reset.high()

else:
    _i2c = _pin_reset = None  # using None for testing without actual hardware device.

bus = adapters.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [6]:
# bus = fx2lp.I2C(as_400KHz = False)

In [7]:
dsp = ADAU1401(bus, pin_reset = _pin_reset)

## Methods Test

In [8]:
from sigma.sigma_dsp.messages import Message, Messages

In [31]:
# load SigmaStudio text data files into "Messages". 

messages = Message.messages_from_SigmaStudio_files(file_NumBytes = 'NumBytes_IC_1.dat', file_TxBuffer = 'TxBuffer_IC_1.dat')

for m in messages[-2:]:
    print(m)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00\x18\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x01'}
{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00\x1c'}


In [10]:
# load messages bytes from binary file

with open('E2Prom.bin', 'br') as f:
    messages_bytes = f.read()

In [11]:
# parse the first segment of data bytes into a "Message".

message, _ = Message.from_bytes(messages_bytes)
print(message)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}


In [32]:
# parse data bytes in to a collections of "Message" object.

messages = Message.messages_from_bytes(messages_bytes)

for m in messages[:5]:
    print(m)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}


In [33]:
dsp.eeprom.find_params_start_address(messages)

70

In [34]:
a = sorted(i for i in range(5))
type(a)

list